In [1]:
%%capture
# run this cell if this package is not installed
# %pip install nbformat

In [2]:
%%capture
%run 1-setup.ipynb

In [3]:
english_word_set = set(words.words())

def is_english_word(word):
    return word.lower() in english_word_set

In [4]:
def preprocess(doc):
    wpt = nltk.WordPunctTokenizer()
    lemmatizer = WordNetLemmatizer()
    stop_words = set(nltk.corpus.stopwords.words('english'))
    
    # remove email addresses
    doc = re.sub(r'\b\S*@\S*\.\S*\b', '', doc)
    
    # remove special characters and digits, retaining only words with letters
    doc = re.sub(r'[^\w\s]', '', doc)
    
    # lowercase and strip
    doc = doc.lower()
    doc = doc.strip()
    
    # remove brackets of any kind
    doc = re.sub(r'[(){}[\]]', '', doc)
    
    # remove punctuation
    doc = doc.translate(str.maketrans("", "", string.punctuation))

    # retain only English words
    doc = ' '.join(word for word in doc.split() if is_english_word(word))
    
    # tokenize document
    tokens = wpt.tokenize(doc)
    
    # determine POS of the tokens
    pos_tags = pos_tag(tokens)
    
    # # map POS tags to WordNet POS tags
    # tag_map = {
    #     'N': wordnet.NOUN,
    #     'V': wordnet.VERB,
    #     'R': wordnet.ADV,
    #     'J': wordnet.ADJ
    # }

    # # lemmatize the tokens
    # lemmatized_tokens = [lemmatizer.lemmatize(token, tag_map.get(pos[0], wordnet.NOUN)) for token, pos in pos_tags]
    
    # keep only nouns
    lemmatized_tokens = [lemmatizer.lemmatize(token, wordnet.NOUN) for token, pos in pos_tags if pos.startswith('N')]
    
    # filter stopwords out of lemmatized tokens 
    filtered_tokens = [token for token in lemmatized_tokens if token not in stop_words]
    
    # recreate the document
    doc = ' '.join(filtered_tokens)
    
    return doc

In [5]:
df_clean = df['Content'].apply(preprocess)

In [6]:
df_clean.iloc[1000]

'surface organization research center canada article hi surface anyone topic information surface thanks help surface plane cut cylinder cone plane course surface patch vanishing curvature ie matrix patch surface plane sphere earth way curvature look book geometry book author geometry year publisher note enjoy university university st canada'